In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5,6"
import pandas as pd 
pd.set_option('display.max_columns', None)
import optuna
import math
import numpy as np 
import seaborn as sns 
from autogluon.common.savers import save_pd
from autogluon.tabular import TabularPredictor, TabularDataset
import warnings 
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [5]:
tr_d = TabularDataset('C:/Users/16010/Desktop/Deep learning from Scratch/kaggle-S4E7/data/train.csv')
te_d = TabularDataset('C:/Users/16010/Desktop/Deep learning from Scratch/kaggle-S4E7/data/test.csv')
tr_d = tr_d.drop("id",axis=1)
uid = te_d['id']
te_d = te_d.drop("id",axis=1)

In [6]:
tr_d.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [7]:
te_d.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


In [10]:
params = {
    
    "XGB": {
        
        "colsample_bytree": 0.516652313273348,
        "enable_categorical": True,
        "learning_rate": 0.015,
        "max_depth": 9,
        "min_child_weight": 0.8567068904025429,
        "max_bin": 262143,
        "n_estimators": 10000,
        "tree_method": "gpu_hist",  # 使用GPU
        "predictor": "gpu_predictor" 
          },
    
    'GBM': {
          
        'n_estimators': 1190,
        'learning_rate': 0.22952000374471332,
        'max_depth': 13, 
        'reg_alpha': 8.200152384535924,
        'reg_lambda': 4.285393733702208, 
        'num_leaves': 100, 
        'subsample': 0.6497981764924947, 
        'colsample_bytree': 0.37368304607248115,
        'device': 'gpu'  
           } 

}

In [11]:
print(tr_d.info())
print(tr_d.describe())

<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Gender                object 
 1   Age                   int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           object 
 6   Vehicle_Damage        object 
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               int64  
 10  Response              int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 965.5+ MB
None
                Age  Driving_License   Region_Code  Previously_Insured  \
count  1.150480e+07     1.150480e+07  1.150480e+07        1.150480e+07   
mean   3.838356e+01     9.980220e-01  2.641869e+01        4.629966e-01   
std    1.499346e+01     4.443120e-02  1.299159e+01        4.986289e-01   
min    2.000000e+01     0.000000e+00  0.000000e+00     

In [12]:
included_models = ["GBM", "XGB",]

PREDICTOR_AUTO = TabularPredictor(label = 'Response'
                                  ,eval_metric = 'roc_auc',).fit(
        
        train_data = tr_d,
        
        hyperparameters = params,
        
        time_limit = 3600*6,  
        
        presets='best_quality',
    
        included_model_types = included_models,
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240705_004629"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       14.94 GB / 31.84 GB (46.9%)
Disk Space Avail:   81.74 GB / 729.84 GB (11.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdo

(_ray_fit pid=48956) [1000]	valid_set's binary_logloss: 0.250467


(raylet) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
(raylet)   warnings.warn(
(_ray_fit pid=48956) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=48956)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument") [repeated 2x across cluster]
(_ray_fit pid=48956) 1 warning generated. [repeated 55x across cluster]


(_ray_fit pid=2328) [1000]	valid_set's binary_logloss: 0.250479


(raylet) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
(raylet)   warnings.warn(
(_ray_fit pid=27564) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
(_ray_fit pid=27564)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
(_ray_fit pid=53708) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
(_ray_fit pid=53708)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
(_ray_fit pid=27564) [LightGBM] [Fatal] Check failed: (best_split_info.right_count) > (0) at D:\a\1\s\lightgbm-python\src\treelearner\serial_tree_learner.cpp, line 856 .
(_ray_fit pid=27564) 


...

(raylet) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
(raylet)   warnings.warn(
(_ray_fit pid=29076) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
(_ray_fit pid=29076)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


(_ray_fit pid=49780) [1000]	valid_set's binary_logloss: 0.250577


(_ray_fit pid=29076) [LightGBM] [Fatal] Check failed: (best_split_info.right_count) > (0) at D:\a\1\s\lightgbm-python\src\treelearner\serial_tree_learner.cpp, line 856 .
(_ray_fit pid=29076) 
(_ray_fit pid=29076) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu
(_ray_fit pid=29076) C:\Users\16010\.conda\envs\3.10forkaggleS4E7\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
(_ray_fit pid=29076)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


(_ray_fit pid=29076) [1000]	valid_set's binary_logloss: 0.250877


(_dystack pid=41496) 	0.8813	 = Validation score   (roc_auc)
(_dystack pid=41496) 	1021.0s	 = Training   runtime
(_dystack pid=41496) 	83.33s	 = Validation runtime
(_dystack pid=41496) Fitting model: XGBoost_BAG_L1 ... Training model for up to 2551.34s of the 4345.72s of remaining time.
(_dystack pid=41496) 	Warning: Not enough memory to safely train model. Estimated to require 20.805 GB out of 12.067 GB available memory (172.416%)... (100.000% of avail memory is the max safe size)
(_dystack pid=41496) 	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.77 to avoid the error)
(_dystack pid=41496) 		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
(_dystack pid=41496) 		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more 

In [13]:
PREDICTOR_AUTO.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.881399,roc_auc,111.559919,1189.294574,1.768074,6.400234,3,True,4
1,LightGBM_BAG_L2,0.881374,roc_auc,109.791845,1182.894341,4.607082,102.935292,2,True,3
2,LightGBM_BAG_L1,0.881345,roc_auc,105.184763,1079.959049,105.184763,1079.959049,1,True,1
3,WeightedEnsemble_L2,0.881345,roc_auc,106.979437,1080.385511,1.794674,0.426462,2,True,2


In [19]:
import lightgbm as lgb
print('LightGBM version:', lgb.__version__)
print('LightGBM built with GPU support:', lgb.basic._is_lib_gpu())

LightGBM version: 4.3.0


AttributeError: module 'lightgbm.basic' has no attribute '_is_lib_gpu'

In [20]:
predictions = PREDICTOR_AUTO.predict(te_d)

In [23]:
submission = pd.DataFrame({
    'id': uid,  # 确保您的测试数据中包含'id'列
    'Response': predictions
})
submission.to_csv('C:/Users/16010/Desktop/Deep learning from Scratch/kaggle-S4E7/submission/submission.csv', index = False)